In [ ]:
# 필수 패키지 설치
!pip install openai pandas numpy sqlite3
!pip install sentence-transformers  # 실험용 추가 패키지
!pip install scikit-learn  # 유사도 계산용

# 프로젝트 루트 디렉토리 추가 (실제 프로젝트 코드 import용)
import sys
import os
sys.path.append(os.path.abspath('../backend'))

print("환경설정 완료!")


In [ ]:
import openai
import re
import json
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass

# 실험용 ParsedInput 클래스 (실제 프로젝트의 models.py와 동일)
@dataclass
class ParsedInput:
    """
    파싱된 입력 데이터 클래스
    - 실제 프로젝트: backend/app/models.py
    - 필드 추가/수정 시 실제 models.py에도 반영 필요
    """
    scenario: str
    location: Optional[str] = None
    equipment_type: Optional[str] = None
    status_code: Optional[str] = None
    priority: str = "일반작업"
    itemno: Optional[str] = None
    confidence: float = 0.0
    
    def to_dict(self):
        return {
            'scenario': self.scenario,
            'location': self.location,
            'equipment_type': self.equipment_type,
            'status_code': self.status_code,
            'priority': self.priority,
            'itemno': self.itemno,
            'confidence': self.confidence
        }

print("ParsedInput 클래스 정의 완료")


In [ ]:
class ExperimentalInputParser:
    """
    실험용 InputParser 클래스
    - 실제 프로젝트: backend/app/agents/parser.py
    - 개선된 로직을 실제 parser.py에 복사/치환하여 반영
    
    실험 가능한 요소들:
    1. LLM 모델 변경 (GPT-4, GPT-3.5, HuggingFace 등)
    2. 프롬프트 엔지니어링
    3. 시나리오 판단 로직 개선
    4. 정보 추출 알고리즘 변경
    """
    
    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        """
        실험용 파서 초기화
        
        Args:
            api_key: OpenAI API 키
            model: 사용할 LLM 모델 (실험 시 교체 가능)
        """
        openai.api_key = api_key
        self.model = model
        
        # ITEMNO 패턴 (실험 시 수정/추가 가능)
        self.itemno_patterns = [
            r'\b[A-Z]{2,4}-\d{5}\b',  # 예: RFCC-00123
            r'\b[A-Z]-\w+\b',         # 예: Y-MV1035
            r'\b\d{5}-[A-Z]{2}-\d+"-[A-Z]\b',  # 예: 44043-CA1-6"-P
        ]
        
        # 우선순위 키워드 (실험 시 수정/추가 가능)
        self.priority_keywords = {
            "긴급작업": ["긴급", "긴급작업", "urgent", "emergency"],
            "우선작업": ["우선", "우선작업", "priority", "high"],
            "일반작업": ["일반", "일반작업", "normal", "regular"]
        }
    
    def parse_input(self, user_input: str) -> ParsedInput:
        """
        사용자 입력 파싱 (실험용)
        
        Args:
            user_input: 사용자 입력 메시지
            
        Returns:
            ParsedInput: 파싱된 구조화된 데이터
        """
        try:
            # 1. 시나리오 판단 (실험 시 로직 개선 가능)
            scenario = self._determine_scenario(user_input)
            
            if scenario == "S1":
                # 시나리오 1: 자연어 작업 요청
                return self._parse_scenario_1(user_input)
            elif scenario == "S2":
                # 시나리오 2: ITEMNO 기반 요청
                return self._parse_scenario_2(user_input)
            else:
                # 기본 시나리오
                return self._parse_default_scenario(user_input)
                
        except Exception as e:
            print(f"파싱 오류: {e}")
            return ParsedInput(scenario="S1", confidence=0.0)
    
    def _determine_scenario(self, user_input: str) -> str:
        """
        시나리오 판단 로직 (실험 시 개선 가능)
        
        실험 아이디어:
        1. 키워드 기반 → ML 분류 모델로 변경
        2. 정규식 패턴 개선
        3. LLM 기반 시나리오 판단
        """
        # ITEMNO 패턴 확인
        for pattern in self.itemno_patterns:
            if re.search(pattern, user_input, re.IGNORECASE):
                return "S2"
        
        # ITEMNO 텍스트 확인
        if re.search(r'ITEMNO\s*\d+', user_input, re.IGNORECASE):
            return "S2"
        
        # 자연어 작업 요청 키워드 확인
        keywords = ['고장', '누설', '작동불량', '점검', '정비', '압력', '온도', '밸브', '펌프', '탱크']
        if any(keyword in user_input for keyword in keywords):
            return "S1"
        
        return "default"
    
    def _parse_scenario_1(self, user_input: str) -> ParsedInput:
        """
        시나리오 1 파싱: 자연어 작업 요청 (실험용)
        
        실험 아이디어:
        1. 프롬프트 엔지니어링으로 추출 정확도 향상
        2. Few-shot learning 적용
        3. Chain-of-Thought 프롬프팅
        """
        # LLM 프롬프트 생성 (실험 시 개선 가능)
        prompt = self._create_scenario_1_prompt(user_input)
        
        try:
            # LLM 호출 (실험 시 모델/파라미터 변경 가능)
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "당신은 설비관리 시스템의 입력 분석 전문가입니다."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,  # 실험 시 조정 가능
                max_tokens=500
            )
            
            result_text = response.choices[0].message.content.strip()
            parsed_data = self._parse_llm_response(result_text)
            
            return ParsedInput(
                scenario="S1",
                location=parsed_data.get('location'),
                equipment_type=parsed_data.get('equipment_type'),
                status_code=parsed_data.get('status_code'),
                priority=parsed_data.get('priority', '일반작업'),
                confidence=parsed_data.get('confidence', 0.8)
            )
            
        except Exception as e:
            print(f"LLM 호출 오류: {e}")
            return ParsedInput(scenario="S1", confidence=0.0)

print("ExperimentalInputParser 클래스 정의 완료")


In [ ]:
    def _create_scenario_1_prompt(self, user_input: str) -> str:
        """
        시나리오 1용 프롬프트 생성 (실험 시 개선 가능)
        
        실험 아이디어:
        1. Few-shot examples 추가
        2. Chain-of-Thought 프롬프팅
        3. 구조화된 출력 포맷 개선
        """
        return f"""
다음 설비관리 요청을 분석하여 구조화된 정보를 추출해주세요.

입력: {user_input}

추출할 정보:
1. 위치/공정 (location): 예) No.1 PE, RFCC, 석유제품배합/저장 등
2. 설비유형 (equipment_type): 예) Pressure Vessel, Motor Operated Valve, Pump 등  
3. 현상코드 (status_code): 예) 고장, 누설, 작동불량 등
4. 우선순위 (priority): 긴급작업, 우선작업, 일반작업 중 하나

JSON 형태로 응답해주세요:
{{
    "location": "추출된 위치",
    "equipment_type": "추출된 설비유형", 
    "status_code": "추출된 현상코드",
    "priority": "추출된 우선순위",
    "confidence": 0.0-1.0
}}

예시:
입력: "1PE 압력베젤 고장"
출력: {{"location": "No.1 PE", "equipment_type": "Pressure Vessel", "status_code": "고장", "priority": "일반작업", "confidence": 0.9}}
"""
    
    def _parse_scenario_2(self, user_input: str) -> ParsedInput:
        """시나리오 2 파싱: ITEMNO 기반 요청"""
        # ITEMNO 추출
        itemno_match = re.search(r'ITEMNO\s*(\d+)', user_input, re.IGNORECASE)
        if itemno_match:
            itemno = itemno_match.group(1)
        else:
            # 패턴 매칭으로 ITEMNO 추출
            for pattern in self.itemno_patterns:
                match = re.search(pattern, user_input)
                if match:
                    itemno = match.group(0)
                    break
            else:
                itemno = None
        
        return ParsedInput(
            scenario="S2",
            itemno=itemno,
            confidence=0.9 if itemno else 0.3
        )
    
    def _parse_default_scenario(self, user_input: str) -> ParsedInput:
        """기본 시나리오 파싱"""
        return ParsedInput(
            scenario="default",
            confidence=0.5
        )
    
    def _parse_llm_response(self, response_text: str) -> Dict:
        """
        LLM 응답 파싱 (실험 시 개선 가능)
        
        실험 아이디어:
        1. JSON 파싱 오류 처리 개선
        2. 정규식 기반 백업 파싱 추가
        3. 신뢰도 점수 계산 로직 개선
        """
        try:
            # JSON 파싱 시도
            if '{' in response_text and '}' in response_text:
                json_start = response_text.find('{')
                json_end = response_text.rfind('}') + 1
                json_str = response_text[json_start:json_end]
                return json.loads(json_str)
            else:
                # JSON이 없는 경우 기본값 반환
                return {}
        except json.JSONDecodeError:
            print(f"JSON 파싱 실패: {response_text}")
            return {}

# 추가 메서드들을 ExperimentalInputParser 클래스에 동적으로 추가
ExperimentalInputParser._create_scenario_1_prompt = _create_scenario_1_prompt
ExperimentalInputParser._parse_scenario_2 = _parse_scenario_2
ExperimentalInputParser._parse_default_scenario = _parse_default_scenario
ExperimentalInputParser._parse_llm_response = _parse_llm_response

print("InputParser 메서드 추가 완료")


In [ ]:
# 실험 설정
OPENAI_API_KEY = "your-api-key-here"  # 실제 실험 시 본인의 API 키 입력

# 실험할 모델들 (교체 가능)
MODELS_TO_TEST = [
    "gpt-3.5-turbo",
    "gpt-4",
    # "gpt-4-turbo-preview"  # 새로운 모델 테스트 시 추가
]

# 테스트 입력 데이터
TEST_INPUTS = [
    "1PE 압력베젤 고장",
    "RFCC 열교환기 누설 긴급",
    "컨베이어 벨트 교체 필요",
    "ITEMNO 44043-CA1-6\"-P 작업상세",
    "Y-MV1035 모터밸브 작동불량",
    "석유제품배합 저장탱크 압력상승"
]

print("실험 설정 완료!")
print(f"테스트할 모델 수: {len(MODELS_TO_TEST)}")
print(f"테스트 입력 수: {len(TEST_INPUTS)}")


In [ ]:
# 단일 모델 테스트
def test_single_model(api_key: str, model: str = "gpt-3.5-turbo"):
    """단일 모델로 파서 테스트"""
    print(f"\n=== {model} 테스트 ===")
    
    if api_key == "your-api-key-here":
        print("API 키를 설정해주세요!")
        return
    
    parser = ExperimentalInputParser(api_key, model)
    
    for i, test_input in enumerate(TEST_INPUTS, 1):
        print(f"\n{i}. 입력: {test_input}")
        try:
            result = parser.parse_input(test_input)
            print(f"   결과: {result}")
        except Exception as e:
            print(f"   오류: {e}")

# 실행 예시 (API 키 설정 후 주석 해제)
# test_single_model(OPENAI_API_KEY, "gpt-3.5-turbo")

print("단일 모델 테스트 함수 준비 완료!")


In [ ]:
# HuggingFace 기반 파서 (실험용)
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    print("transformers 라이브러리가 없습니다. 설치하려면:")
    print("pip install transformers torch")
    TRANSFORMERS_AVAILABLE = False

class HuggingFaceInputParser:
    """
    HuggingFace 모델 기반 파서 (실험용)
    - OpenAI 대신 오픈소스 모델 사용
    - 온프레미스 배포 가능
    - 비용 절감 효과
    """
    
    def __init__(self, model_name: str = "microsoft/DialoGPT-medium"):
        """
        HuggingFace 모델 초기화
        
        추천 모델들:
        - "microsoft/DialoGPT-medium": 대화형 모델
        - "facebook/blenderbot-400M-distill": 대화 모델
        - "google/flan-t5-base": 지시 따르기 모델
        """
        if not TRANSFORMERS_AVAILABLE:
            raise ImportError("transformers 라이브러리를 설치해주세요")
        
        self.model_name = model_name
        print(f"모델 로딩 중: {model_name}")
        
        try:
            # 파이프라인 생성 (실험 시 다른 모델로 교체 가능)
            self.generator = pipeline(
                "text-generation",
                model=model_name,
                max_length=512,
                temperature=0.1
            )
            print("모델 로딩 완료!")
        except Exception as e:
            print(f"모델 로딩 실패: {e}")
            self.generator = None
    
    def parse_input(self, user_input: str) -> ParsedInput:
        """HuggingFace 모델로 입력 파싱"""
        if not self.generator:
            return ParsedInput(scenario="S1", confidence=0.0)
        
        # 프롬프트 생성
        prompt = f"""
설비관리 요청을 분석해주세요:
입력: {user_input}

위치: 
설비유형:
현상코드:
우선순위:
"""
        
        try:
            # 모델 실행
            outputs = self.generator(prompt, max_new_tokens=100, do_sample=False)
            result_text = outputs[0]['generated_text']
            
            # 간단한 파싱 (실험 시 개선 가능)
            return self._parse_hf_response(result_text, user_input)
            
        except Exception as e:
            print(f"HuggingFace 모델 오류: {e}")
            return ParsedInput(scenario="S1", confidence=0.0)
    
    def _parse_hf_response(self, response: str, original_input: str) -> ParsedInput:
        """HuggingFace 모델 응답 파싱 (실험용)"""
        # 간단한 키워드 기반 파싱 (실험 시 개선 가능)
        location = None
        equipment_type = None
        status_code = None
        
        # 키워드 매칭
        if "PE" in original_input:
            location = "No.1 PE"
        elif "RFCC" in original_input:
            location = "RFCC"
        
        if "압력" in original_input or "베젤" in original_input:
            equipment_type = "Pressure Vessel"
        elif "밸브" in original_input:
            equipment_type = "Valve"
        elif "펌프" in original_input:
            equipment_type = "Pump"
        
        if "고장" in original_input:
            status_code = "고장"
        elif "누설" in original_input:
            status_code = "누설"
        elif "작동불량" in original_input:
            status_code = "작동불량"
        
        return ParsedInput(
            scenario="S1",
            location=location,
            equipment_type=equipment_type,
            status_code=status_code,
            priority="일반작업",
            confidence=0.7
        )

if TRANSFORMERS_AVAILABLE:
    print("HuggingFace 파서 클래스 준비 완료!")
else:
    print("transformers 라이브러리를 설치하면 HuggingFace 모델을 사용할 수 있습니다.")
